In [1]:
from datetime import datetime

import pandas as pd
import numpy as np

In [43]:
df = pd.read_csv('sales.csv')

In [14]:
df.groupby('team').count()

,name,sales
team,,
blue,4,4
green,2,2
red,4,4


In [15]:
df['team'].value_counts()

red      4
blue     4
green    2
Name: team, dtype: int64

In [19]:
df.groupby('team').size()

team
blue     4
green    2
red      4
dtype: int64

In [60]:
# to do the same operation above but save it as a new col in the same df, use transform
# you have to groupby and then select only that column and then transform to create the new col in the original dataframe.
df['team_count_agg'] = df \
    .groupby('team')['team'] \
    .transform('count')

df['team_sales_agg'] = df \
    .groupby('team')['sales'] \
    .transform('sum')

# to group by whole dataset make this dummy variable and delete it after
df['dummy'] = 1
df['total_sales_agg'] = df \
    .groupby('dummy')['sales'] \
    .transform('sum')
df = df.drop('dummy', axis = 1)

df_sales = df.query('team_sales_agg > 200').sort_values('team_sales_agg', ascending = False) # query the team_sales col for values > 200 and sort descending

certain_name = df[df['name'].str.contains('tom')] # return the row if name contains the string 'tom'

df['total_sales_after_tax'] = df['total_sales_agg'] * .8
df['scrape_date'] = datetime.now().date()
df['scrape_ts'] = datetime.now()